In [15]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [16]:
retrieved_data = requests.get(**{
    "url" : "https://api.pushshift.io/reddit/submission/search/",
    "params": {
        'after': 1585699199, #get dates after March 31, 2020
        'before' : 1588291200, #get dates before May 1, 2020
        'subreddit': 'dlsu',
        'size': 500,
        'fields': ["author", "subreddit", "created_utc", "num_comments", "score", "title", "selftext"]
    }
}).json()["data"]

In [17]:
def format_data():
    data = {
        "author": [],
        "subreddit": [],
        "created_at": [],
        "num_of_comments": [],
        "score": [],
        "title": [],
        "description": []
    }
    for reddit in retrieved_data:
        data["author"].append(reddit["author"])
        data["subreddit"].append(reddit["subreddit"])
        data["title"].append(reddit["title"])
        data["description"].append(reddit["selftext"])
        data["score"].append(reddit["score"])
        data["num_of_comments"].append(reddit["num_comments"])
        data["created_at"].append(reddit["created_utc"])
    return data

df = pd.DataFrame(data=format_data())
df["created_at"] = pd.to_datetime(df["created_at"],unit="s",origin="unix")
df["day_of_week"] = df["created_at"].dt.day_name()

In [18]:
total_submission_cnt = len(df)
text = f"Total Submission Count: {total_submission_cnt}"
print(f"\n\n{text}")


print("\n\n\n---------- Highest / Most Submissions with idxmax() (one entry only) ---------")
author_with_most_submissions = df["author"].mode()[0]
author_submission_count = len(df[df["author"] == author_with_most_submissions])
text = f"Author with Most Submissions: {author_with_most_submissions} ({author_submission_count} submissions)"
print(text)
submission_with_highest_score = df.loc[df["score"].idxmax()]
text = f"Submission with highest score: '{submission_with_highest_score['title']}' by {submission_with_highest_score['author']} (has a score of {submission_with_highest_score['score']})"
print(text)
submission_with_highest_comms = df.loc[df["num_of_comments"].idxmax()]
text = f"Submission with highest comments: '{submission_with_highest_comms['title']}' by {submission_with_highest_comms['author']} ({submission_with_highest_comms['num_of_comments']} comments)"
print(text)
df_agg = df.groupby(['day_of_week'])["day_of_week"].count()
max_day_index = df_agg.idxmax()
text = f"Day of week with most submissions: {max_day_index} ({df_agg.loc[max_day_index]} submissions)"
print(text)


print("\n\n\n---------- Highest / Most Submissions with max() (possibly multiple entries) ---------")

df_authors_freq = df.groupby(['author'])['author'].agg('count').to_frame('frequency').reset_index()
# print(df_authors_freq)
max_cnt = df_authors_freq['frequency'].max()
print(f"\nAuthor(s) with the highest number of submissions ({max_cnt} submissions):")
i = 1
for _, row in df_authors_freq[df_authors_freq['frequency'] == max_cnt].iterrows():
    print(f"{i}. {row['author']}")
    i += 1
    
highest_score = df["score"].max()
print(f"\nSubmission(s) with highest score (score of {highest_score}):")
i = 1
for _, row in df[df['score'] == highest_score].iterrows():
    print(f"{i}. '{row['title']}' by {row['author']}")
    i += 1

highest_comms = df["num_of_comments"].max()
print(f"\nSubmission(s) with highest comments ({highest_comms} comments):")
i = 1
for _, row in df[df['num_of_comments'] == highest_comms].iterrows():
    print(f"{i}. '{row['title']}' by {row['author']}")
    i += 1

df_day_of_week_freq = df.groupby(['day_of_week'])["day_of_week"].agg('count').to_frame('frequency').reset_index()
max_cnt = df_day_of_week_freq['frequency'].max()
print(f"\nDay(s) of week with most submissions ({max_cnt} submissions):")
i = 1
for _, row in df_day_of_week_freq[df_day_of_week_freq['frequency'] == max_cnt].iterrows():
    print(f"{i}. {row['day_of_week']}")
    i += 1

print("\n\n")



Total Submission Count: 100



---------- Highest / Most Submissions with idxmax() (one entry only) ---------
Author with Most Submissions: camnacio_13 (11 submissions)
Submission with highest score: 'Opportunities in abroad' by Cheezyreine (has a score of 9)
Submission with highest comments: 'Laptop' by vgtrxx (14 comments)
Day of week with most submissions: Sunday (18 submissions)



---------- Highest / Most Submissions with max() (possibly multiple entries) ---------

Author(s) with the highest number of submissions (11 submissions):
1. camnacio_13

Submission(s) with highest score (score of 9):
1. 'Opportunities in abroad' by Cheezyreine

Submission(s) with highest comments (14 comments):
1. 'Laptop' by vgtrxx

Day(s) of week with most submissions (18 submissions):
1. Sunday



